In [12]:
import pandas as pd
data = pd.read_csv(r"C:\Users\HP\Videos\Github\Hybrid_perdiction_model\apple_stock_data.csv")
data.head()

,Date,Adj Close,Close,High,Low,Open,Volume
0,2023-11-02 00:00:00+00:00,176.665985,177.570007,177.779999,175.460007,175.520004,77334800
1,2023-11-03 00:00:00+00:00,175.750671,176.649994,176.820007,173.350006,174.240005,79763700
2,2023-11-06 00:00:00+00:00,178.317520,179.229996,179.429993,176.210007,176.380005,63841300
3,2023-11-07 00:00:00+00:00,180.894333,181.820007,182.440002,178.970001,179.179993,70530000
4,2023-11-08 00:00:00+00:00,181.958893,182.889999,183.449997,181.589996,182.350006,49340300


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       252 non-null    object 
 1   Adj Close  252 non-null    float64
 2   Close      252 non-null    float64
 3   High       252 non-null    float64
 4   Low        252 non-null    float64
 5   Open       252 non-null    float64
 6   Volume     252 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 13.9+ KB


In [14]:
data["Date"] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
data = data[['Close']]

In [15]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data['Close'] = scaler.fit_transform(data)

In [16]:
import numpy as np

def create_sequences(data, seq_lenght=60):
    X, y = [], []
    for i in range(len(data) - seq_lenght):
        X.append(data[i:i+seq_lenght])
        y.append(data[i+seq_lenght])
    return np.array(X), np.array(y)

seq_lenght = 60
X,y = create_sequences(data['Close'].values, seq_lenght)

In [17]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

153
39
153
39


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
lstm_model.add(LSTM(50))
lstm_model.add(Dense(1))

In [10]:
lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
5/5 [==============================] - 21s 139ms/step - loss: 0.3037
Epoch 2/20
5/5 [==============================] - 1s 109ms/step - loss: 0.0797
Epoch 3/20
5/5 [==============================] - 0s 80ms/step - loss: 0.0456
Epoch 4/20
5/5 [==============================] - 1s 155ms/step - loss: 0.0212
Epoch 5/20
5/5 [==============================] - 1s 98ms/step - loss: 0.0256
Epoch 6/20
5/5 [==============================] - 1s 104ms/step - loss: 0.0194
Epoch 7/20
5/5 [==============================] - 1s 100ms/step - loss: 0.0150
Epoch 8/20
5/5 [==============================] - 0s 94ms/step - loss: 0.0147
Epoch 9/20
5/5 [==============================] - 1s 96ms/step - loss: 0.0119
Epoch 10/20
5/5 [==============================] - 0s 97ms/step - loss: 0.0119
Epoch 11/20
5/5 [==============================] - 1s 99ms/step - loss: 0.0109
Epoch 12/20
5/5 [==============================] - 0s 99ms/step - loss: 0.0107
Epoch 13/20
5/5 [==============================] - 0s 9

#### 2 Linear regression model

In [18]:
# Lagged data features
data['Lag_1'] = data['Close'].shift(1)
data['Lag_2'] = data['Close'].shift(2)
data['Lag_3'] = data['Close'].shift(3)
data = data.dropna()

In [19]:
X_lin = data[['Lag_1', 'Lag_2', 'Lag_3']]
y_lin = data['Close']

X_train_lin, X_test_lin = X_lin[:train_size], X_lin[train_size:]
y_train_lin, y_test_lin = y_lin[:train_size], y_lin[train_size:]


In [ ]:
from sklearn.linear_model import LinearRegression

lin_model = LinearRegression()
lin_model.fit(X_train_lin, y_train_lin)